In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
import math
import re
from collections import Counter

In [111]:
amazon=pd.read_csv('amz_com-ecommerce_sample.csv',encoding = 'unicode_escape')
amazon.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [3]:
amazon.shape

(20000, 15)

In [112]:
flipkart=pd.read_csv("flipkart_com-ecommerce_sample.csv")
flipkart.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
flipkart.shape

(20000, 15)

In [113]:
df=pd.DataFrame({'ProductName in Amazon':amazon['product_name'],
                 'Retail Price (Amazon)':amazon['retail_price'],
                 'Discounted Price (Amazon)':amazon['discounted_price'],
                 'ProductName in Flipkart':flipkart['product_name'],
                 'Retail Price (Flipkart)':flipkart['retail_price'],
                 'Discounted Price (Flipkart)':flipkart['discounted_price'],
                })
df

,ProductName in Amazon,Retail Price (Amazon),Discounted Price (Amazon),ProductName in Flipkart,Retail Price (Flipkart),Discounted Price (Flipkart)
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0
2,AW Bellies,991,551,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0
...,...,...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,WallDesign Small Vinyl Sticker,1500.0,730.0
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1499.0,1199.0


#### Cosine similarity with sparse_dot_topn:

In [114]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    text_words = re.sub(r'[^a-zA-Z0-9\s]',' ',str(text)).split()
    return Counter(text_words)

In [93]:
series_amz[11002]

'jlrs bumper case for acerâ\xa0iconia w3-810|acerâ\xa0iconia w4-820'

In [57]:
text_to_vector(series_amz[11002])

Counter({'jlrs': 1,
         'bumper': 1,
         'case': 1,
         'for': 1,
         'acer': 2,
         'iconia': 2,
         'w3': 1,
         '810': 1,
         'w4': 1,
         '820': 1})

In [58]:
text_to_vector(series_flpkrt[11002])

Counter({'jlrs': 1,
         'bumper': 1,
         'case': 1,
         'for': 1,
         'acer': 2,
         'iconia': 2,
         'w3': 1,
         '810': 1,
         'w4': 1,
         '820': 1})

In [115]:
similarity=[]
series_amz = pd.Series(df['ProductName in Amazon']).str.lower()
series_flpkrt = pd.Series(df['ProductName in Flipkart']).str.lower()
for i in range(len(series_amz)):
    vector1 = text_to_vector(series_amz[i])
    vector2 = text_to_vector(series_flpkrt[i])
    cosine = get_cosine(vector1, vector2)
    similarity.append(cosine)

In [116]:
df['Similarity']=np.round(similarity,2)

In [117]:
df[df.Similarity < 0.75]

,ProductName in Amazon,Retail Price (Amazon),Discounted Price (Amazon),ProductName in Flipkart,Retail Price (Flipkart),Discounted Price (Flipkart),Similarity
18997,S TOES BELLIES,1198,666,Small Toes Bellies,1200.0,599.0,0.67


In [118]:
matched_df = df[df.Similarity > 0.75]

In [119]:
matched_df.drop('Similarity',axis=1,inplace=True)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [120]:
matched_df

,ProductName in Amazon,Retail Price (Amazon),Discounted Price (Amazon),ProductName in Flipkart,Retail Price (Flipkart),Discounted Price (Flipkart)
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0
2,AW Bellies,991,551,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0
...,...,...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,WallDesign Small Vinyl Sticker,1500.0,730.0
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1499.0,1199.0


In [108]:
matched_df.to_excel('Product_similarity.xls')